In [0]:
%run ../config/load_config

In [0]:
target_table = "arrival_event_geo_sv"
silver_table_path = get_storage_path("silver", target_table)

# Ensure silver table
spark.sql(f"""
CREATE TABLE IF NOT EXISTS {catalog}.{schema_silver}.{target_table}(
    arrival_event_id STRING,
    line_id STRING,
    naptan_id STRING,
    expected_arrival TIMESTAMP,
    station_name STRING,
    time_to_live TIMESTAMP,
    event_timestamp TIMESTAMP,
    is_service_disrupted BOOLEAN,
    borough STRING
)
USING DELTA
LOCATION '{silver_table_path}'
""")

# Join arrival events and stops enriched with borough to derive table
spark.sql(f"""
MERGE INTO {catalog}.{schema_silver}.{target_table} AS target
USING (
    SELECT
        al.arrival_event_id,
        al.line_id,
        al.naptan_id,
        al.station_name,
        al.expected_arrival,
        al.time_to_live,
        al.event_timestamp,
        al.is_service_disrupted,
        sg.borough_name AS borough
    FROM {catalog}.{schema_silver}.arrival_events_sv al
    LEFT JOIN {catalog}.{schema_silver}.stops_geo_sv sg
        ON al.naptan_id = sg.naptan_id
) AS source
ON target.arrival_event_id = source.arrival_event_id
   AND target.event_timestamp = source.event_timestamp
WHEN MATCHED THEN UPDATE SET *
WHEN NOT MATCHED THEN INSERT *
""")